<a href="https://colab.research.google.com/github/pnkjjha/BuildNExperiment/blob/main/Structured_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=acdb90badda32647122a3796c756237cc5c88a80a3ba5d6ca17a6a1bc06f117d
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [106]:
from pyspark.sql import SparkSession
from pandas import json_normalize
import pyspark.sql.functions as F
import pyspark.sql.types as T
import random
import requests
import json
import os
import pandas as pd

In [3]:
spark=SparkSession.builder.appName('test').getOrCreate()

In [ ]:
# url = "https://api.carbonintensity.org.uk/intensity"

# while switch:
#   response = requests.get(url)
#   json_normalize(response.json())
#   message = response.json()
#   json_normalize(message['data']).rename(columns={"intensity.forecast": "intensity_forecast",\
#                                                   "intensity.actual": "intensity_actual",\
#                                                   "intensity.percentile": "intensity_percentile",\
#                                                   "intensity.index": "intensity_index"\
#                                                   }).to_csv("intensity_"+str(random.randint(1,10000))+".csv", index=False)


In [107]:
# read lines from file
def flat_json(line):
  line_json = json.loads(line)
# json normalize
  flatline = pd.json_normalize(line_json,sep="_")
  return flatline


In [109]:
s = requests.Session()

i = 0
url = "https://stream.wikimedia.org/v2/stream/recentchange"
with s.get(url, stream=True) as resp:
  for line in resp.iter_lines(1024,decode_unicode=True):
    # Check if line has some bytes in it
    if line:
      # convert line into json
      # get data lines only
      if line[:4] == "data":
          # print(json.loads(line[5:]))
          with open("recentChanges"+str(random.randint(1,10000))+".json","a",encoding="utf-8") as f:
              flat_json(line[5:]+ os.linesep).to_json(f,orient="records",lines=True,index=False)
              print(f.name)
          i = i + 1
    if i>=10 :
      f.close()
      break

recentChanges2403.json
recentChanges8870.json
recentChanges7804.json
recentChanges6230.json
recentChanges9992.json
recentChanges9153.json
recentChanges1131.json
recentChanges7099.json
recentChanges2339.json
recentChanges9738.json


In [112]:
%rm -f *.json

In [119]:
%rm -rf Out*

In [98]:
spark.read.schema(cSchema).load("recentChanges.json").show()

TypeError: Object of type StructField is not JSON serializable

In [64]:
iSchema = spark.read.load(".", format="json").schema

In [95]:
#create Dataframe representing the stream of input lines from connection to localhost:9999
# intensity = spark.readStream.schema(iSchema).load("recentChanges.json")
intensity = spark.readStream.schema(cSchema).load(".")
intensity

TypeError: Object of type StructField is not JSON serializable

In [93]:
# Assuming 'intensity' is your streaming DataFrame
myquery = intensity.groupBy("type").count()\
  .withWatermark("meta.dt", "10 seconds") \
  .writeStream.format("json")\
  .option("path", "Output/result.json")\
  .outputMode("append")\
  .option("checkpointLocation", "checkpoint_dir1")\
  .trigger(processingTime='5 seconds')\
  .start().awaitTermination()


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `meta`.`dt` cannot be resolved. Did you mean one of the following? [`type`, `count`].;
'EventTimeWatermark 'meta.dt, 10 seconds
+- Aggregate [type#1207], [type#1207, count(1) AS count#2221L]
   +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@4a650a73,parquet,List(),Some(StructType(StructField($schema,StringType,true),StructField(bot,BooleanType,true),StructField(comment,StringType,true),StructField(id,LongType,true),StructField(length,StructType(StructField(new,LongType,true),StructField(old,LongType,true)),true),StructField(log_action,StringType,true),StructField(log_action_comment,StringType,true),StructField(log_id,LongType,true),StructField(log_params,ArrayType(StringType,true),true),StructField(log_type,StringType,true),StructField(meta,StructType(StructField(domain,StringType,true),StructField(dt,StringType,true),StructField(id,StringType,true),StructField(offset,Long

In [82]:
spark.read.schema("Type STRING, value INTEGER").csv("Output/result.json").show()

+----+-----+
|Type|value|
+----+-----+
+----+-----+



In [83]:
myquery.stop()